In [2]:
import pandas as pd
import csv
from datetime import date,datetime
from pandas.tseries.offsets import BDay, BMonthEnd
import string
import openpyxl
import os

In [3]:
yDay = date.today()-BMonthEnd(1)
date1=yDay.strftime("%Y%m")
date2=yDay.strftime("%Y%m%d")
tPath="Trades.xlsx"

In [4]:
def ib():
    reportPath="U10289655_{}_{}.csv".format(date1,date1)            
    columns = list(string.ascii_uppercase)[:17]
    Trades=pd.read_csv(reportPath,header=None, names=columns)        
    Trades=Trades[Trades['A']=='Trades'][Trades['C']=='Order'].loc[:,['D','E','F','G','H','I','L']].reset_index().drop("index",axis=1)
    Trades.columns=['product','currency','symbol','date','quantity','price','comms']
    Trades['symbol']=Trades['symbol']+Trades['currency'].map(lambda x: " HK" if x=="HKD" else "")
    Trades['symbol']=Trades['symbol'].map(lambda x: x.replace("CN",'XU').replace("24","4") if "CN" and "24" in x else x)
    Trades['product']=Trades['product'].map(lambda x: x.replace('Stocks','EQ').replace('Futures','Index Future'))
    Trades['date']=Trades['date'].map(lambda x: datetime.strptime(x[0:10],'%Y-%m-%d'))
    Trades['quantity']=Trades['quantity'].map(lambda x: float(x.replace(",",'')))
    Trades['price']=Trades['price'].astype(float)
    Trades['comms']=Trades['comms'].map(lambda x:abs(float(x)))
    Trades['side']=Trades['quantity'].map(lambda x: 'BUY' if x>0 else 'SHORT')
    Trades['quantity']=Trades['quantity'].map(abs)
    
    wb=openpyxl.load_workbook(tPath)
    ws=wb['IB']
    max_row = ws.max_row
    for row in range(2, max_row + 1):
        ws.cell(row=row, column=16).value=None
    startRow=2
    for i in range(0,Trades.shape[0]):
        eachLowList=Trades.iloc[i,:].tolist()
        ws.cell(row=startRow+i, column=5).value=eachLowList[0]
        ws.cell(row=startRow+i, column=24).value=eachLowList[1]
        ws.cell(row=startRow+i, column=18).value=eachLowList[2]
        ws.cell(row=startRow+i, column=16).value=eachLowList[3]
        ws.cell(row=startRow+i, column=22).value=eachLowList[4]
        ws.cell(row=startRow+i, column=25).value=eachLowList[5]
        ws.cell(row=startRow+i, column=30).value=eachLowList[6]
        ws.cell(row=startRow+i, column=21).value=eachLowList[7]
    wb.save(tPath)
    wb.close() 
    return Trades

In [5]:
def clsa():
    reportPath="OrderFills_CITIC_Mth-LeverageHover4piOFC-PS.{}.csv".format(date2) 
    trades=pd.read_csv(reportPath)
    trades=trades.loc[:,['Trade Date','Ticker','Counterparty Buy/Sell','Local Gross Price','Quantity / Exchange-traded Contract','FX Rate','Swap Net Notional']]
    trades.columns=['Date','symbol','side','gross_price','qty','fx','net_amount']#amened  
    trades['Date']=trades['Date'].map(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    
    wb=openpyxl.load_workbook(tPath)
    ws=wb['CLSA']
    max_row = ws.max_row
    for row in range(2, max_row + 1):
        ws.cell(row=row, column=16).value=None
    startRow=2
    for i in range(0,trades.shape[0]):
        eachLowList=trades.iloc[i,:].tolist()
        ws.cell(row=startRow+i, column=16).value=eachLowList[0]
        ws.cell(row=startRow+i, column=18).value=eachLowList[1]
        ws.cell(row=startRow+i, column=21).value=eachLowList[2]
        ws.cell(row=startRow+i, column=25).value=eachLowList[3]
        ws.cell(row=startRow+i, column=23).value=eachLowList[4]
        ws.cell(row=startRow+i, column=26).value=eachLowList[5]
        #ws.cell(row=startRow+i, column=30).value=eachLowList[6]
        #ws.cell(row=startRow+i, column=21).value=eachLowList[7]
    wb.save(tPath)
    wb.close() 

    return trades

In [6]:
ib()

C:\Users\Ning\AppData\Local\Temp\ipykernel_18072\3273866025.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Trades=Trades[Trades['A']=='Trades'][Trades['C']=='Order'].loc[:,['D','E','F','G','H','I','L']].reset_index().drop("index",axis=1)


,product,currency,symbol,date,quantity,price,comms,side
0,Index Future,USD,CNF25,2025-01-08,2.0,12907.000000,4.065307,BUY
1,Index Future,USD,CNF25,2025-01-08,1.0,12910.000000,2.032653,BUY
2,Index Future,USD,CNF25,2025-01-08,3.0,12906.000000,6.097960,SHORT
3,Index Future,USD,CNF25,2025-01-13,1.0,12682.000000,2.032682,BUY
4,Index Future,USD,CNF25,2025-01-13,1.0,12682.000000,2.032682,BUY
5,Index Future,USD,CNF25,2025-01-13,2.0,12682.000000,4.065363,BUY
6,Index Future,USD,CNF25,2025-01-13,4.0,12740.500000,8.130726,SHORT
7,Index Future,USD,CNF25,2025-01-15,1.0,12946.000000,2.032661,BUY
8,Index Future,USD,CNF25,2025-01-15,2.0,12944.500000,4.065321,BUY
9,Index Future,USD,CNF25,2025-01-15,9.0,12939.888889,18.293946,BUY


In [7]:
clsa()

,Date,symbol,side,gross_price,qty,fx,net_amount
0,2025-01-02,601899 CH,SELL,15.330043,-280000.0,7.3242,-5.855817e+05
1,2025-01-02,2899 HK,BUY,14.304076,314000.0,1.0000,4.497701e+06
2,2025-01-02,1548 HK,SELL,9.943333,-6000.0,1.0000,-5.957703e+04
3,2025-01-03,601899 CH,SELL,15.707779,-280000.0,7.3501,-5.978963e+05
4,2025-01-03,2899 HK,BUY,14.451410,312000.0,1.0000,4.515085e+06
